In [1]:
import numpy as np
import pandas as pd

def calculate_hz(lum, teff):
    """
    Calculate the Habitable Zone for a given Stellar Effective Temperature (Teff) using the empirical formula.

    Parameters:
        lum (float or array): Stellar Luminosity in Solar Units.
        teff (float or array): Stellar Effective Temperature in Kelvin.

    Returns:
        tuple: A tuple containing the inner and outer radii of the Habitable Zone in Astronomical Units (AU).
    """
    # Constants for the empirical formula
    coeff = {"inner": [1.0140, 8.1774e-5, 1.7063e-9, -4.3241e-12, -6.6462e-16],
             "outer": [0.3438, 5.8942e-5, 1.6558e-9, -3.0045e-12, -5.2983e-16]}
    teff_star = teff - 5780

    s_inner = (coeff["inner"][0] + coeff["inner"][1] * teff_star + coeff["inner"][2] * teff_star**2 +
               coeff["inner"][3] * teff_star**3 + coeff["inner"][4] * teff_star**4)
    s_outer = (coeff["outer"][0] + coeff["outer"][1] * teff_star + coeff["outer"][2] * teff_star**2 +
               coeff["outer"][3] * teff_star**3 + coeff["outer"][4] * teff_star**4)

    hz_inner = np.sqrt(lum / s_inner)
    hz_outer = np.sqrt(lum / s_outer)
    return hz_inner, hz_outer

# Load the CSV file
df = pd.read_csv("TOI_2025.01.10_gaia_3_predicted_multipl_sptype_APCPC+CP+KP.csv")

# Filter the data based on the given criteria
filtered_df = df[(df['Prediction'] == 1) & (df['lum_flame'].notna())]

# Use the 'mass_flame' column directly for host star mass
filtered_df['Mass'] = filtered_df['mass_flame']

# Calculate Distance to star (R_in_AU)
# Only calculate for rows where 'mass_flame' is not NaN
filtered_df['a_orbit'] = np.where(
    filtered_df['Mass'].notna(),
    np.cbrt((filtered_df['pl_orbper'] / 365.256)**2 * filtered_df['Mass']),
    np.nan
)

# Calculate the HZ inner and outer radii
hz_inner, hz_outer = calculate_hz(filtered_df['lum_flame'], filtered_df['st_teff'])

# Add the new columns to the DataFrame
filtered_df['hz_inner'] = hz_inner
filtered_df['hz_outer'] = hz_outer

# Determine if the planet is in the habitable zone
def habitable_zone(row):
    if pd.isna(row['Mass']):
        return 'no mass_flame'
    elif row['a_orbit'] >= row['hz_inner'] and row['a_orbit'] <= row['hz_outer']:
        return 'in'
    elif row['a_orbit'] < row['hz_inner'] and (row['hz_inner'] - row['a_orbit']) <= 0.1:
        return 'close'
    elif row['a_orbit'] > row['hz_outer'] and (row['a_orbit'] - row['hz_outer']) <= 0.1:
        return 'close'
    else:
        return 'out'

filtered_df['Habitable_Zone'] = filtered_df.apply(habitable_zone, axis=1)

# Save the updated DataFrame to a new CSV file
new_file_path = 'TOI_2025.01.10_gaia_3_predicted+CP+KP_multipl_sptype_hz_2.csv'
filtered_df.to_csv(new_file_path, index=False)

print(f"Updated CSV saved to {new_file_path}")

Updated CSV saved to TOI_2025.01.10_gaia_3_predicted+CP+KP_multipl_sptype_hz_2.csv


C:\Users\huangm\AppData\Local\Temp\ipykernel_6092\1271784793.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Mass'] = filtered_df['mass_flame']
C:\Users\huangm\AppData\Local\Temp\ipykernel_6092\1271784793.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['a_orbit'] = np.where(
C:\Users\huangm\AppData\Local\Temp\ipykernel_6092\1271784793.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [9]:
# NOT USED as using mass_flame directly rather than calculating mass based on lum

import pandas as pd
import numpy as np

# Load the CSV file
df = pd.read_csv("TOI_2025.01.10_gaia_3_predicted_multipl_sptype_APCPC+CP+KP.csv")

# Filter the data based on the given criteria
filtered_df = df[(df['Prediction'] == 1) & (df['lum_flame'].notna())]

# Calculate host star mass
def calculate_mass(lum):
    m1 = np.power((lum / 0.23), 1/2.3)
    m2 = np.power(lum, 1/4)
    m3 = np.power((lum / 1.4),1/3.5)
    m4 = lum / 32000
    if m1 < 0.43:
        return m1
    elif 0.43 <= m2 < 2:
        return m2
    elif 2 <= m3< 55:
        return m3
    else:
        return m4

filtered_df['Mass'] = filtered_df['lum_flame'].apply(calculate_mass)

# Calculate Distance to star (R_in_AU)
filtered_df['a_orbit'] = np.cbrt((filtered_df['pl_orbper'] / 365.256)**2 * filtered_df['Mass'])

# Calculate the HZ inner and outer radii
hz_inner, hz_outer = calculate_hz(filtered_df['lum_flame'], filtered_df['st_teff'])

# Add the new columns to the DataFrame
filtered_df['hz_inner'] = hz_inner
filtered_df['hz_outer'] = hz_outer

# Determine if the planet is in the habitable zone
def habitable_zone(row):
    if row['a_orbit'] >= row['hz_inner'] and row['a_orbit'] <= row['hz_outer']:
        return 'in'
    elif row['a_orbit'] < row['hz_inner'] and (row['hz_inner'] - row['a_orbit']) <= 0.1:
        return 'close'
    elif row['a_orbit'] > row['hz_outer'] and (row['a_orbit'] - row['hz_outer']) <= 0.1:
        return 'close'
    else:
        return 'out'

filtered_df['Habitable_Zone'] = filtered_df.apply(habitable_zone, axis=1)

# Save the updated DataFrame to a new CSV file
new_file_path = 'TOI_2025.01.10_gaia_3_predicted+CP+KP_multipl_sptype_hz.csv'
filtered_df.to_csv(new_file_path, index=False)

print(f"Updated CSV saved to {new_file_path}")

Updated CSV saved to TOI_2025.01.10_gaia_3_predicted+CP+KP_multipl_sptype_hz.csv


C:\Users\huangm\AppData\Local\Temp\ipykernel_14480\701183102.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Mass'] = filtered_df['lum_flame'].apply(calculate_mass)
C:\Users\huangm\AppData\Local\Temp\ipykernel_14480\701183102.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['a_orbit'] = np.cbrt((filtered_df['pl_orbper'] / 365.256)**2 * filtered_df['Mass'])
C:\Users\huangm\AppData\Local\Temp\ipykernel_14480\701183102.py:34: SettingWithCopyWarning: 
A value is trying to be se